# Lecture Notebook 02 — Non-Benzodiazepine Hypnotics (Dynamic)

**Interactive assets: PubChem 2D/3D, target maps, SAR & metabolism tables**

This notebook is designed for use in a public GitHub repository. It includes optional dynamic content:
- PubChem structure images (via PUG REST)
- 2D/3D structure retrieval (SDF) and 3D visualization (py3Dmol)
- Interaction/target maps (matplotlib/networkx)
- SAR and metabolism “soft spot” tables (pandas)

> **Note:** This notebook makes **live web requests** to PubChem when executed. If you run in an offline environment, use the provided caching cells to save images/SDF locally.


## ⚠️ Instructor-Run Notebook (Student-Consumed Outputs)

This notebook is intended to be **executed by the instructor/TA only**. Students will consume the generated outputs (figures, tables, maps) via slides, PDFs, or Canvas.

**Outputs produced:**
- `figures/` (slide-ready images)
- `data/` (CSV tables + manifest)

**Suggested workflow:** Run all cells → commit outputs → distribute the rendered notebook or exported HTML/PDF.


## 0. Setup

Run the cell below once per environment. It installs lightweight libraries used for visualization.


In [ ]:
import sys, subprocess, textwrap
def pip_install(pkgs):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + pkgs)

# Uncomment if needed
# pip_install(['requests', 'pandas', 'matplotlib', 'networkx', 'py3Dmol'])

import requests, pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path

try:
    import py3Dmol
except Exception:
    py3Dmol = None

DATA_DIR = Path('data')
FIG_DIR  = Path('figures')
DATA_DIR.mkdir(exist_ok=True)
FIG_DIR.mkdir(exist_ok=True)
print('Ready. py3Dmol:', bool(py3Dmol))


## 1. Utility functions (PubChem PUG REST)

These helpers fetch:
- **Compound CID** from name
- **PNG structure image**
- **SDF** (2D/3D coordinates)

They also support caching to `data/` and `figures/` so the repository can be built reproducibly.


In [ ]:
PUBCHEM = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug'

def cid_from_name(name: str) -> int:
    url = f"{PUBCHEM}/compound/name/{requests.utils.quote(name)}/cids/JSON"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return int(r.json()['IdentifierList']['CID'][0])

def fetch_png_by_cid(cid: int, outpath: Path, size=600):
    url = f"{PUBCHEM}/compound/cid/{cid}/PNG?image_size={size}x{size}"
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    outpath.write_bytes(r.content)
    return outpath

def fetch_sdf_by_cid(cid: int, outpath: Path, record_type='3d'):
    # record_type: '2d' or '3d'
    url = f"{PUBCHEM}/compound/cid/{cid}/SDF?record_type={record_type}"
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    outpath.write_text(r.text)
    return outpath

def get_drug_assets(drug: str):
    cid = cid_from_name(drug)
    png_path = FIG_DIR / f"{drug.replace(' ', '_')}_PubChem.png"
    sdf_path = DATA_DIR / f"{drug.replace(' ', '_')}_3d.sdf"
    if not png_path.exists():
        fetch_png_by_cid(cid, png_path)
    if not sdf_path.exists():
        # fall back to 2d if 3d not available
        try:
            fetch_sdf_by_cid(cid, sdf_path, record_type='3d')
        except Exception:
            fetch_sdf_by_cid(cid, sdf_path, record_type='2d')
    return cid, png_path, sdf_path


## 2. Drug list for this lecture

Edit this list to match your lecture scope. Keep names aligned with your slide deck tables for consistency.


In [ ]:
DRUGS = ['Zolpidem', 'Zaleplon', 'Eszopiclone']
DRUGS

## 3. Pull PubChem images + cache

This will download structure PNGs and SDF files for each drug.


In [ ]:
assets = []
for d in DRUGS:
    try:
        cid, png, sdf = get_drug_assets(d)
        assets.append({'Drug': d, 'CID': cid, 'PNG': str(png), 'SDF': str(sdf)})
    except Exception as e:
        assets.append({'Drug': d, 'CID': None, 'PNG': None, 'SDF': None, 'Error': str(e)})

assets_df = pd.DataFrame(assets)
assets_df


## 4. Display PubChem images (2D)


In [ ]:
from PIL import Image

for _, row in assets_df.iterrows():
    if row.get('PNG'):
        img = Image.open(row['PNG'])
        display(img)
        print(row['Drug'], 'CID:', row['CID'])


## 5. 3D rendering (py3Dmol)

If `py3Dmol` is available, this cell will render the SDF in-browser.


In [ ]:
def show_3d_from_sdf(sdf_path: str, style='stick'):
    if not py3Dmol:
        print('py3Dmol not installed. Run pip_install(["py3Dmol"]) and re-import.')
        return
    sdf = Path(sdf_path).read_text()
    view = py3Dmol.view(width=700, height=450)
    view.addModel(sdf, 'sdf')
    view.setStyle({style:{}})
    view.zoomTo()
    return view.show()

# Example: render the first drug that has an SDF
first = assets_df.dropna(subset=['SDF']).head(1)
if len(first):
    show_3d_from_sdf(first.iloc[0]['SDF'])


## 6. Drug–Target interaction map (lecture-aligned)

Edit the `edges` list to match the curated targets and binding sites used in your lecture.


In [ ]:
edges = [('Zolpidem', 'GABA_A (BZD site α–γ)', 'PAM'), ('Zaleplon', 'GABA_A (BZD site α–γ)', 'PAM'), ('Eszopiclone', 'GABA_A (BZD site α–γ)', 'PAM')]
G = nx.DiGraph()
for drug, target, mode in edges:
    G.add_edge(drug, target, label=mode)

plt.figure(figsize=(10,6))
pos = nx.spring_layout(G, seed=7)
nx.draw_networkx_nodes(G, pos, node_size=1200)
nx.draw_networkx_labels(G, pos, font_size=9)
nx.draw_networkx_edges(G, pos, arrows=True)
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
plt.axis('off')
out = FIG_DIR / 'drug_target_map.png'
plt.tight_layout()
plt.savefig(out, dpi=200)
plt.show()
print('Saved:', out)


## 7. SAR and metabolism tables (editable)

Fill in or extend the tables below to match your slide-deck SAR and metabolism narratives. These tables are designed to be exported to CSV for LMS/assessment use.


In [ ]:
sar_df = pd.DataFrame(
    [('Non-BZD scaffold', 'Preserves BZD-site geometry', 'Binds α–γ interface'), ('↑ lipophilicity', '↑ onset; ↑ next-day impairment', 'PK tuning'), ('Polar substitution', '↓ duration', 'Faster clearance'), ('Zaleplon-like design', 'Ultra-short action', 'Middle-of-night dosing')],
    columns=['Structural feature', 'Effect', 'MedChem rationale']
)
met_df = pd.DataFrame(
    [('Zolpidem', 'CYP3A4 oxidation → inactive metabolites', 'Sleep-onset'), ('Zaleplon', 'Aldehyde oxidase major', 'Ultra-short'), ('Eszopiclone', 'CYP3A4/CYP2E1', 'Maintenance; dysgeusia risk')],
    columns=['Drug/Class', 'Major metabolism', 'Clinical implication']
)
display(sar_df)
display(met_df)

sar_df.to_csv(DATA_DIR/'sar_table.csv', index=False)
met_df.to_csv(DATA_DIR/'metabolism_table.csv', index=False)
print('Saved:', DATA_DIR/'sar_table.csv', 'and', DATA_DIR/'metabolism_table.csv')


## 8. Export summary assets

This cell creates a simple manifest you can commit to GitHub so all generated assets are tracked.


In [ ]:
manifest = {
  'lecture': 'L02',
  'assets': assets_df.to_dict(orient='records'),
  'generated': [str(FIG_DIR/'drug_target_map.png'), str(DATA_DIR/'sar_table.csv'), str(DATA_DIR/'metabolism_table.csv')]
}
import json
Path(DATA_DIR/'manifest.json').write_text(json.dumps(manifest, indent=2))
print('Wrote', DATA_DIR/'manifest.json')


## 9. Instructor Export Pack (Slide-Ready)

This section converts notebook outputs into **slide-ready** assets:
- A single `assets_manifest.json`
- A `captions.md` file with figure captions
- Copies key figures into `figures/slide_ready/`


In [ ]:
import json, shutil
from pathlib import Path

SLIDE_DIR = Path('figures')/'slide_ready'
SLIDE_DIR.mkdir(parents=True, exist_ok=True)

# Collect candidate figures
candidates = []
for p in Path('figures').glob('*.png'):
    if p.name != '':
        candidates.append(p)
for p in Path('figures').glob('*.jpg'):
    candidates.append(p)

# Copy to slide_ready (preserve names)
copied = []
for p in candidates:
    dest = SLIDE_DIR/p.name
    shutil.copy2(p, dest)
    copied.append(str(dest))

# Build captions (edit as needed)
captions = []
for p in candidates:
    captions.append({'file': f'figures/slide_ready/{p.name}', 'caption': f'Figure: {p.stem.replace('_',' ').replace('-',' ')} (auto-generated caption — please edit).'} )

Path(SLIDE_DIR/'captions.md').write_text('\n'.join([f"- ![{c['caption']}]({c['file']})" for c in captions]))

# Save a consolidated manifest
manifest_path = Path('data')/'assets_manifest.json'
manifest = {
  'generated_on': '2026-01-03',
  'slide_ready_files': copied,
  'notes': 'Auto-generated by Instructor Export Pack. Edit captions.md as needed.'
}
manifest_path.write_text(json.dumps(manifest, indent=2))
print('Wrote:', manifest_path)
print('Slide-ready figures:', len(copied))
print('Captions:', SLIDE_DIR/'captions.md')
